## Tensorflow Dataset API

##### 목표

1. tf.data를 사용하여 메모리에서 데이터를 읽는 방법
2. 훈련시 tf.data를 사용하는 방법
3. tf.data를 사용하여 디스크에서 데이터를 읽는 방법
4. 기능 엔지니어링(일괄 처리, 셔플링 등)을 사용하여 프로덕션 입력 파이프라인을 작성하는 방법 

In [ ]:
import json
import math
import os
from pprint import pprint

import numpy as np
import tensorflow as tf
print(tf.version.VERSION)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# dataset 생성

N_POINTS = 10
# The .constant() method will creates a constant tensor from a tensor-like object.
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10

In [ ]:
# create_dataset() 함수 생성
# epoch, batch
def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

In [ ]:
BATCH_SIZE = 3
EPOCH = 2

dataset = create_dataset(X, Y, epochs=EPOCH, batch_size=BATCH_SIZE)

for i, (x, y) in enumerate(dataset):
    print("x:", x.numpy(), "y:", y.numpy())
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE

In [ ]:
# 손실함수 정의 loss_mse() 
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)


# 기울기 계산하는 함수 정의 compute_gradients()
def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

In [ ]:
# 모델 훈련하기
# TODO 2
EPOCHS = 250
BATCH_SIZE = 2
LEARNING_RATE = .02

MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dataset = create_dataset(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE)

for step, (X_batch, Y_batch) in enumerate(dataset):

    dw0, dw1 = compute_gradients(X_batch, Y_batch, w0, w1)
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)

    if step % 100 == 0:
        loss = loss_mse(X_batch, Y_batch, w0, w1)
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))
        
assert loss < 0.0001
assert abs(w0 - 2) < 0.001
assert abs(w1 - 10) < 0.001

In [ ]:
# 데이터의 속성 이름 지정
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]

In [ ]:
# 함수 정의
def create_dataset(pattern):
    return tf.data.experimental.make_csv_dataset(
        pattern, 1, CSV_COLUMNS, DEFAULTS)


tempds = create_dataset('../toy_data/taxi-train*')
print(tempds)

In [ ]:
# 데이터 확인
for data in tempds.take(2):
    pprint({k: v.numpy() for k, v in data.items()})
    print("\n")

In [ ]:
UNWANTED_COLS = ['pickup_datetime', 'key']

# Let's define the features_and_labels() method
# TODO 4a
def features_and_labels(row_data):
# The .pop() method will return item and drop from frame. 
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label

In [ ]:
for row_data in tempds.take(2):
    features, label = features_and_labels(row_data)
    pprint(features)
    print(label, "\n")
    assert UNWANTED_COLS[0] not in features.keys()
    assert UNWANTED_COLS[1] not in features.keys()
    assert label.shape == [1]